# Практическое занятие по методам трекинга объектов на основе модели Yolo

## Подготовка окружения для работы с Yolo

Установка пакета Ultralytics для работы с моделью Yolo.

In [ ]:
!pip install ultralytics

Загрузка весовых коэффициентов предобученной модели.

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8x.pt

In [ ]:
!ls

## Детекция объектов на изображении

In [ ]:
%%writefile sources.csv
url
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_001.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_002.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_003.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_2_fixed/image_005.jpg

In [ ]:
!mkdir images
!cd images && for line in `cat ../sources.csv`; do wget $line; done

In [ ]:
%%writefile localfiles.csv
images/image_001.jpg
images/image_002.jpg
images/image_003.jpg
images/image_005.jpg

In [ ]:
from ultralytics import YOLO
import plotly.express as plte

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x.pt')
# Define remote image or video URL
source = 'localfiles.csv'
# Run inference on the source
results = model("./images")  # list of Results objects
for res in results:
    plte.imshow(res.plot()).show()

In [ ]:
(results[0].names[int(results[0].boxes.cls[0])], results[0].names[int(results[0].boxes.cls[1])]) #

## Трекинг объектов

In [ ]:
video = "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/video_1.mp4"

# Perform tracking with the model
results = model.track(source=video, show=False)  # Tracking with default tracker

In [ ]:
plte.imshow(results[5].plot())

In [ ]:
"{1:0>5}".format(5, 4)

In [ ]:
!mkdir output
from collections import defaultdict

import cv2
import numpy as np

# Open the video file
# video_path = "tracked_video.mp4"
# cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

i = 0
for frame in results:
    # Get the boxes and track IDs
    boxes = frame.boxes.xywh.cpu()
    annotated_frame = frame.plot()

    if not frame.boxes.id is None:
        track_ids = frame.boxes.id.int().cpu().tolist()

        # Visualize the results on the frame


        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # Display the annotated frame
    cv2.imwrite("output/image{0:0>5}.jpg".format(i), annotated_frame)
    i = i + 1
# Release the video capture object and close the display window
# cap.release()
# cv2.destroyAllWindows()
!ffmpeg -framerate 30 -pattern_type glob -i 'output/*.jpg' -c:v libx264 -pix_fmt yuv420p out.mp4

In [ ]:
frame.boxes